In [7]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [8]:
data = pd.read_csv("Online_retail.csv")
data["InvoiceDate"]=pd.to_datetime(data["InvoiceDate"])
print(data.shape)
print(data.count())
data['Country']=data['Country'].str.capitalize()

(541910, 8)
Invoice        541910
StockCode      541910
Description    540456
Quantity       541910
InvoiceDate    541910
Price          541910
Customer ID    406830
Country        541910
dtype: int64


Description of Columns:
1. Invoice = Unique id given to every transaction.
2. StockCode = Unique Id given to every item.
3. Description = Explain the item.
4. Quantity =  The number of items.
5. InvoiceDate = Day of the transaction
6. Price =  Price of the item
7. Customer ID =  Unique id for every customer
8. Country = Location of transaction.

The Data contains sales by line items. Therefore, feature values are duplicated when analysise. Additionally, it appears
the data contains void transactions and would need to be clean for further analysis.

In [9]:
data_clean = data[(data['Price'] > 0) & (data['Quantity']>0)] # remove all void or sample transactions
data_clean.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United kingdom


In [10]:

def agg_data(df):
    """Group Invoice by customer and sum total quantity and Price"""
    customer = df.groupby(['Invoice','Country','InvoiceDate','Customer ID']).agg({'Quantity': 'count', 'Price': 'sum'}).reset_index()
    return customer

def year_month(df):
    df['InvoiceYearMonth']=df['InvoiceDate'].map(lambda date: 100*date.year+date.month)
    return df

def get_month(x):
    """Get month from date"""
    return dt.datetime(x.year, x.month, 1)

def entry_month(df):
    """Create date format to help calculate retention"""
    df['InvoiceMonth'] = df['InvoiceDate'].apply(get_month)
    grouping = df.groupby('Customer ID')['InvoiceMonth']
    df['CohortMonth'] = grouping.transform('min')
    return df

def group_df(df):
    """Create groups to gain more insight"""
    totalpatients=df.groupby(['InvoiceYearMonth'])['Customer ID'].nunique()
    NewPatient=df.groupby(['CohortMonth']).agg({"Customer ID":pd.Series.nunique})
    revenue=df.groupby(['InvoiceYearMonth']).agg({'Price':'sum'})
    #new_group=pd.concat([totalpatients,NewPatient,revenue],axis=1)
    return totalpatients,NewPatient,revenue

def mtr_cal(df):
    """Creates Matrix to calculate retention"""
    user_purchase=df.groupby(['Customer ID','InvoiceYearMonth'])['Price'].sum().reset_index()
    retention_mtr=pd.crosstab(user_purchase['Customer ID'],user_purchase['InvoiceYearMonth']).reset_index()
    month=retention_mtr.columns[2:]
    return retention_mtr,month

def cal_retention(df):
    """ Calculate retention and creates Dataframe"""
    retention_array = []
    retention_mtr,month = mtr_cal(df)
    for i in range(len(month)-1):
        retention_data={}
        selected_month=month[i+1]
        prev_month=month[i]
        retention_data['InvoiceYearMonth']=int(selected_month)
        retention_data['TotalUserCount']=retention_mtr[selected_month].sum()
        retention_data['RetainedUserCount']=retention_mtr[(retention_mtr[selected_month]>0)&(retention_mtr[prev_month]>0)][selected_month].sum()
        retention_data['New Transaction']=retention_mtr[(retention_mtr[selected_month]>0)&(retention_mtr[prev_month]==0)][selected_month].sum()
        retention_data['Previous Total']= retention_mtr[prev_month].sum()
        retention_array.append(retention_data)
    retention_mtr=pd.DataFrame(retention_array)
    retention_mtr['RetentionRate']=round((retention_mtr['RetainedUserCount']/retention_mtr['Previous Total'])*100,2)#.round(2).astype(str) + '%'
    return retention_mtr

def cal_details(df,df_mtr):
    """Calculate patients from previous months to calculate the actual retention."""
    totalpatients,NewPatient,revenue = group_df(df)
    NewPatient.drop(NewPatient.index[:2], inplace=True)
    revenue.drop(revenue.index[:2],inplace=True)
    complete1 = np.concatenate([df_mtr,NewPatient,revenue],axis=1)
    complete = pd.DataFrame(complete1)
    complete.columns = ['Month', 'Total_Patients', 'Retained_Patients', 'New_Transaction','Previous_Total','Retention_Rate','New_Patients','Revenue']
    complete['Reactivated'] = complete['New_Transaction']-complete['New_Patients']
    complete['Reactivated_Rate'] = round((complete['Reactivated']/complete['Total_Patients'])*100,2)#.round(2).astype(str) + '%'
    complete['New_Patients_Rate'] = round((complete['New_Patients']/complete['Total_Patients'])*100,2)#.round(2).astype(str) + '%'
    complete['Full_Retention'] = round(((complete['Retained_Patients'] + complete['Reactivated'])/complete['Total_Patients'])*100,2)#.round(2).astype(str) + '%'
    complete_retention=complete[['Month', 'Total_Patients','Previous_Total','New_Patients','Retained_Patients','Reactivated',
    'New_Transaction','Reactivated_Rate','New_Patients_Rate','Retention_Rate','Full_Retention','Revenue']]
    complete_retention['Month']=complete_retention['Month'].astype('int64')
    #print(complete_retention)
    return complete_retention

In [11]:
def main():
    data = data_clean
    modified_data = agg_data(data)
    modified_date = year_month(modified_data)
    initial_month = entry_month(modified_date)
    #create_group = group_df(initial_month)
    #retention_matrix = mtr_cal(modified_date)
    contractual_retention = cal_retention(initial_month)
    complete_retention = cal_details(initial_month,contractual_retention)
    #print(pd.DataFrame(complete_retention))
    return complete_retention
    

    


In [12]:
a = main()
a

,Month,Total_Patients,Previous_Total,New_Patients,Retained_Patients,Reactivated,New_Transaction,Reactivated_Rate,New_Patients_Rate,Retention_Rate,Full_Retention,Revenue
0,201102,758.0,741.0,380.0,262.0,116.0,496.0,15.30,50.13,35.36,49.87,63185.760
1,201103,974.0,758.0,452.0,290.0,232.0,684.0,23.82,46.41,38.26,53.59,88563.100
2,201104,856.0,974.0,300.0,304.0,252.0,552.0,29.44,35.05,31.21,64.95,79005.481
3,201105,1056.0,856.0,284.0,368.0,404.0,688.0,38.26,26.89,42.99,73.11,102128.280
4,201106,991.0,1056.0,242.0,410.0,339.0,581.0,34.21,24.42,38.83,75.58,85274.730
5,201107,949.0,991.0,188.0,365.0,396.0,584.0,41.73,19.81,36.83,80.19,75919.901
6,201108,935.0,949.0,169.0,388.0,378.0,547.0,40.43,18.07,40.89,81.93,79423.830
7,201109,1266.0,935.0,299.0,425.0,542.0,841.0,42.81,23.62,45.45,76.38,119053.062
8,201110,1364.0,1266.0,358.0,489.0,517.0,875.0,37.90,26.25,38.63,73.75,165849.980
9,201111,1664.0,1364.0,323.0,622.0,719.0,1042.0,43.21,19.41,45.60,80.59,185522.460
